In [1]:
import glob
import os
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion", "I can't get no relief Business men, they drink my wine", 'Plowman dig my earth', 'None were level on the mind', 'Nobody up at his word', 'Hey, hey No reason to get excited', 'The thief he kindly spoke', 'There are many here among us']


In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sa,mple        sentence."))

<start> this is sa , mple sentence . <end>


In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence.count(" ") > 14: continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 118   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 633 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f467b56cd10>


In [5]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  62 271  27  94 546  20  86 742  90   3   0   0   0]
[ 62 271  27  94 546  20  86 742  90   3   0   0   0   0]


In [6]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 1)

In [7]:
print("Source:", src_input.shape)
print("Target:", tgt_input.shape)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source: (156013, 14)
Target: (156013, 14)
Source Train: (124810, 14)
Target Train: (124810, 14)


In [8]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
487/487 [==============================] - 546s 1s/step - loss: 3.8895 - val_loss: 3.0037
Epoch 2/10
487/487 [==============================] - 517s 1s/step - loss: 2.9018 - val_loss: 2.7855
Epoch 3/10
487/487 [==============================] - 520s 1s/step - loss: 2.6467 - val_loss: 2.6345
Epoch 4/10
487/487 [==============================] - 515s 1s/step - loss: 2.4245 - val_loss: 2.5140
Epoch 5/10
487/487 [==============================] - 515s 1s/step - loss: 2.2033 - val_loss: 2.4212
Epoch 6/10
487/487 [==============================] - 516s 1s/step - loss: 1.9976 - val_loss: 2.3459
Epoch 7/10
487/487 [==============================] - 518s 1s/step - loss: 1.7973 - val_loss: 2.2848
Epoch 8/10
487/487 [==============================] - 514s 1s/step - loss: 1.6154 - val_loss: 2.2387
Epoch 9/10
487/487 [==============================] - 515s 1s/step - loss: 1.4498 - val_loss: 2.2070
Epoch 10/10
487/487 [==============================] - 514s 1s/step - loss: 1.3022 - val_lo

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [12]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you liberian girl , <end> '

In [13]:
generate_text(model, tokenizer, init_sentence="<start> i threw", max_len=20)

'<start> i threw suicides on the private jet <end> '

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i looked", max_len=20)

'<start> i looked at sunrise , <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i trade", max_len=20)

'<start> i trade my <unk> , i m <unk> through the <unk> <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> your stare", max_len=20)

'<start> your stare are wrong <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> where you", max_len=20)

'<start> where you going to run to now ? <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> hey ,", max_len=20)

'<start> hey , hey , hey , oh , oh , oh <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> but here", max_len=20)

'<start> but here s a shotgun <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> love game", max_len=20)

'<start> love game intuition play the cards with spades to start <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> a little", max_len=20)

'<start> a little bit of spanish castle magic <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> cuz we", max_len=20)

'<start> cuz we re almost there <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> give me", max_len=20)

'<start> give me a million reasons <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> my teenage", max_len=20)

'<start> my teenage one , no one <end> '

텍스트 제너레이션 결과 이상한 문장이 생성되기도 하지만 대체적으로 그럴듯하게 생성되었다.